In [1]:
from models import get_depth_estimation_model, get_fire_segmentation_model
import os
import numpy as np

In [2]:
%load_ext tensorboard

In [3]:
import tensorflow as tf
from data_prep import FireData
from keras.callbacks import LearningRateScheduler


def learning_rate_schedule(epoch, learning_rate):
    new_learning_rate = max(learning_rate * .95, 1e-7)
    return new_learning_rate

lr_scheduler = LearningRateScheduler(learning_rate_schedule)

EPOCHS = 2
BATCH_SIZE = 16
LEARNING_RATE = 0.0005
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    amsgrad=False,
)
model = get_fire_segmentation_model()
model(np.random.normal(size=(2, 256, 256, 3)))
if os.path.isfile(model.file_path):
    model.load_weights(model.file_path)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=False),
              metrics=tf.keras.metrics.MeanAbsoluteError())
train_loader = FireData(split='train', batch_size=BATCH_SIZE)
valid_loader = FireData(split='valid', batch_size=BATCH_SIZE)
model.fit(
    train_loader,
    epochs=EPOCHS,
    validation_data=valid_loader,
    validation_steps=len(valid_loader),
    callbacks=[model.checkpoint_callback,
               model.tensor_board_callback,
               lr_scheduler]
)


Epoch 1/2
1373/1373 [==============================] - 257s 183ms/step - loss: 1.5898e-04 - val_loss: 7.8196e-05 - lr: 4.7500e-04
Epoch 2/2
1373/1373 [==============================] - 251s 183ms/step - loss: 6.7247e-05 - val_loss: 6.1514e-05 - lr: 4.5125e-04


In [ ]:
%tensorboard --logdir 'Models/UNet Fire Segmentation/logs'

In [4]:
import tensorflow as tf
from data_prep import DistanceData
from keras.callbacks import LearningRateScheduler


def learning_rate_schedule(epoch, learning_rate):
    new_learning_rate = max(learning_rate * .95, 1e-7)
    return new_learning_rate

lr_scheduler = LearningRateScheduler(learning_rate_schedule)

EPOCHS = 2
BATCH_SIZE = 4
LEARNING_RATE = 0.0005
HEIGHT = 1024 // 8
WIDTH = 768 // 8
optimizer = tf.keras.optimizers.Adam(
    learning_rate=LEARNING_RATE,
    amsgrad=False,
)
model = get_depth_estimation_model()
# Define the loss function
cross_entropy = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction="none"
)
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, amsgrad=False),
              metrics=tf.keras.metrics.MeanAbsoluteError())
train_loader = DistanceData(split='train', batch_size=BATCH_SIZE, height=HEIGHT, width=WIDTH)
valid_loader = DistanceData(split='val', batch_size=BATCH_SIZE, height=HEIGHT, width=WIDTH)
model.fit(
    train_loader,
    epochs=EPOCHS,
    validation_data=valid_loader,
    validation_steps=len(valid_loader),
    callbacks=[model.checkpoint_callback,
               model.tensor_board_callback,
               lr_scheduler]
)


Epoch 1/2
6364/6364 [==============================] - 1717s 269ms/step - loss: 0.5443 - val_loss: 0.4186 - lr: 4.7500e-04
Epoch 2/2
6364/6364 [==============================] - 1701s 267ms/step - loss: 0.5015 - val_loss: 0.3996 - lr: 4.5125e-04


In [ ]:
%tensorboard --logdir 'Models/UNet Depth Estimation/logs'